In [4]:
from parse_documents import *

filepath = 'upload/Jean-Baptiste_Perronneau_-_Magdaleine_Pinceloup_de_la_Grange,_née_de_Parseval.jpg'  # Replace with the path of your file
# filepath = "upload/World_Energy_By_Country_And_Region_1965_to_2023.csv"
# filepath = 'upload/Diabetic-retinopathy-identification-using-parallel-convo_2023_Expert-Systems.pdf'
parse_document(filepath)



<class 'dict'>
dict_keys(['metadata', 'content', 'status'])
{'Number of Tables': '4 Huffman tables', 'File Modified Date': 'Sun Sep 15 00:33:54 +00:00 2024', 'Compression Type': 'Baseline', 'Data Precision': '8 bits', 'X-TIKA:Parsed-By-Full-Set': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.image.JpegParser'], 'Number of Components': '3', 'tiff:ImageLength': '623', 'resourceName': "b'Jean-Baptiste_Perronneau_-_Magdaleine_Pinceloup_de_la_Grange,_n\\xc3\\xa9e_de_Parseval.jpg'", 'Component 2': 'Cb component: Quantization table 1, Sampling factors 1 horiz/1 vert', 'Component 1': 'Y component: Quantization table 0, Sampling factors 2 horiz/2 vert', 'Image Height': '623 pixels', 'Image Width': '500 pixels', 'File Size': '72495 bytes', 'Component 3': 'Cr component: Quantization table 1, Sampling factors 1 horiz/1 vert', 'X-TIKA:Parsed-By': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.image.JpegParser'], 'X-TIKA:parse_time_millis': '11', 'X-TIKA:embedded

In [8]:
from tika import parser

# Open your file in binary mode to get its raw bytes
with open(filepath, 'rb') as f:
    content = f.read()

parser = parser.from_bytes(content, metadata=True)

# The 'filename' field of parser.metadata should now contain the original name with preserved encoding.
print(parser.metadata['filename'])

AttributeError: module 'tika.parser' has no attribute 'from_bytes'

In [5]:
import chardet

# Open your file in binary mode to get its raw bytes
with open(filepath, 'rb') as f:
    content = f.read()

parser = tika.parser.from_bytes(content, metadata=True)
encoding_guess = chardet.detect(parser.get_content())

if encoding_guess['confidence'] > 0.9:  # Confidence threshold can be adjusted based on your needs
    # Use the detected encoding to re-interpret the file name correctly.
    original_filename = parser.metadata['resourceName'].decode(encoding_guess['encoding'])
else:
    # For cases where detection is uncertain or fails, you might consider falling back to UTF-8 (or another default).
    original_filename = parser.metadata['resourceName'].decode('utf-8', errors='replace')

print(original_filename)

ModuleNotFoundError: No module named 'chardet'

In [ ]:
from tika import parser

def parse_document(filepath):
    try:
        parsed_document = parser.from_file(filepath)
        
        # Check if parsing was successful
        if 'status' in parsed_document and parsed_document['status'] == 200:
            print(f"Parsing successful: {parsed_document['status']}")
        else:
            print(f"Parsing failed with status: {parsed_document.get('status')}")
            return
        
        # Print the type and keys of the parsed document
        print(type(parsed_document))
        print(parsed_document.keys())

        # Extract and print metadata if available
        metadata = parsed_document.get('metadata', {})
        if metadata:
            print("Metadata:")
            print(metadata)
        else:
            print("No metadata found.")
        
        # Print content if needed (commented to avoid large outputs)
        # print(parsed_document.get('content', 'No content available'))
        
    except FileNotFoundError:
        print(f"Error: File '{filepath}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
parse_document("upload/Jean-Baptiste_Perronneau_-_Magdaleine_Pinceloup_de_la_Grange,_née_de_Parseval.jpg")


In [ ]:
from tika import parser

def parse_document(filepath):
    try:
        parsed_document = parser.from_file(filepath)
        
        # Check if parsing was successful
        if 'status' in parsed_document and parsed_document['status'] == 200:
            print(f"Parsing successful: {parsed_document['status']}")
        else:
            print(f"Parsing failed with status: {parsed_document.get('status')}")
            return
        
        # Extract and print list of keys in the parsed document
        document_keys = list(parsed_document.keys())
        print("Keys in the parsed document:")
        print(document_keys)

        # Extract metadata keys if available
        metadata = parsed_document.get('metadata', {})
        if metadata:
            metadata_keys = list(metadata.keys())
            print("Keys in metadata:")
            print(metadata_keys)
        else:
            print("No metadata found.")

    except FileNotFoundError:
        print(f"Error: File '{filepath}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
parse_document("upload/Jean-Baptiste_Perronneau_-_Magdaleine_Pinceloup_de_la_Grange,_née_de_Parseval.jpg")


In [ ]:
import os
import json
from tika import parser
from ipfs_functions import *

def parse_documents_in_directory(directory_path):
    temp_dir = "tmp" # Get the system's temporary directory
    print(f"Saving metadata files in temporary directory: {temp_dir}")

    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        upload_file_to_ipfs(directory_path + "/" + filename)
        
        # Check if it's a file and not a directory
        if os.path.isfile(file_path):
            try:
                parsed_document = parser.from_file(file_path)
                
                # Check if parsing was successful
                if 'status' in parsed_document and parsed_document['status'] == 200:
                    metadata = parsed_document.get('metadata', {})
                    print(metadata)
                    upload_json_to_ipfs(metadata)
                    
                    # Save metadata as JSON
                    if metadata:
                        json_filename = f"{os.path.splitext(filename)[0]}.json"
                        json_path = os.path.join(temp_dir, json_filename)

                        with open(json_path, 'w') as json_file:
                            json.dump(metadata, json_file, indent=4)
                        
                        print(f"Metadata saved for '{filename}' as '{json_filename}'")
                    else:
                        print(f"No metadata found for '{filename}'")
                else:
                    print(f"Parsing failed for '{filename}' with status: {parsed_document.get('status')}")
            
            except Exception as e:
                print(f"An error occurred with file '{filename}': {e}")

# Example usage
parse_documents_in_directory("upload")


In [2]:
import os
import json
from tika import parser
from ipfs_functions import *

def parse_documents_in_directory(directory_path):
    

    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        print(file_path)
        upload_file_to_ipfs(file_path)
        
        # Check if it's a file and not a directory
        if os.path.isfile(file_path):
            try:
                parsed_document = parser.from_file(file_path)
                
                # Check if parsing was successful
                if 'status' in parsed_document and parsed_document['status'] == 200:
                    metadata = parsed_document.get('metadata', {})
                    upload_json_to_ipfs(metadata)
                    
                else:
                    print(f"Parsing failed for '{filename}' with status: {parsed_document.get('status')}")
            
            except Exception as e:
                print(f"An error occurred with file '{filename}': {e}")

# Example usage
parse_documents_in_directory("upload")


ic| result['Name']: 'assets.csv'
    result['Hash']: 'QmWRMwc5taM4BaapEhLPpmZVPrN7WmxoEyyaQvoSgUqfWW'
ic| file_metadata_CID: 'QmaufPiaidvwUQz9gWxZXsBdMsyAEDd3MyyuLF4Y8btnzo'
ic| result['Name']: 'Diabetic-retinopathy-identification-using-parallel-convo_2023_Expert-Systems.pdf'
    result['Hash']: 'QmT7ximApXtExAedhzUHGHDwRismRzuKykpH4dgTvGrNZs'


upload/assets.csv
upload/Diabetic-retinopathy-identification-using-parallel-convo_2023_Expert-Systems.pdf


ic| file_metadata_CID: 'QmUaGWKsvakqW7Vu2pBpEsjApc6UYd2yrsLaE7P73ESW9c'
ic| result['Name']: 'Munafò et al. - 2022 - The reproducibility debate is an opportunity, not .pdf'
    result['Hash']: 'QmdiRawzVNUiB28ENKQ7WefeFLEJ1xMjsJjwtHL2jnJ9xW'
ic| file_metadata_CID: 'QmW21d1oEvzw3cjEPkxMDZ1uhxgQ3LrPEfhLyp4d6ZJ2yA'
ic| result['Name']: 'assets.json'
    result['Hash']: 'QmZ9tmsiUTt22eyHqkA84wns29sWwZ94LGHBZzUjbaGwFu'
ic| file_metadata_CID: 'QmdDXTjH47hf9X6LkSVb8pxWnV3CbLfNzm69Bvo8QBiKWK'
ic| result['Name']: 'Mel-Spectrogram-based-advanced-deep-temporal-clusterin_2023_Expert-Systems-w.pdf'
    result['Hash']: 'QmXquoApMLCcauNkz4tYw1Sp2tsqZ4edv1YtnQi5pmbw6C'


upload/Munafò et al. - 2022 - The reproducibility debate is an opportunity, not .pdf
upload/assets.json
upload/Mel-Spectrogram-based-advanced-deep-temporal-clusterin_2023_Expert-Systems-w.pdf


ic| file_metadata_CID: 'QmPgVNGQYSacGPMMPdxnZUySX19hLoa7hiHVeF59bZwsbE'
ic| result['Name']: 'Aeroacoustic-airfoil-shape-optimization-enhance_2023_Expert-Systems-with-App.pdf'
    result['Hash']: 'QmXh37WXcrLXkJX2cAPjbPdKZ2cxJXD58XX6eU1Zc41ka4'


...

ic| file_metadata_CID: 'QmcKLJJjPzK3DPbKuogfvKhfmbUfdvRmzDfV43M7nhwKL4'
ic| result['Name']: 'COVID19-MLSF--A-multi-task-learning-based-stock-market_2023_Expert-Systems-w.pdf'
    result['Hash']: 'QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW'
ic| file_metadata_CID: 'QmYNLkMVn78vk3hb47SpaqMsTmsCbjHxUfqUKbZm43WsmV'
ic| result['Name']: 'World_Energy_By_Country_And_Region_1965_to_2023.csv'
    result['Hash']: 'QmSSY49SnmbCZ3oSaTki7CYZe1ZaWZfE1CsWHpt8Ge7acJ'
ic| file_metadata_CID: 'QmPWyTKJ5kArkhtfU3sFqeBe5gzPCj8NEC964av3r3rmMc'


upload/COVID19-MLSF--A-multi-task-learning-based-stock-market_2023_Expert-Systems-w.pdf
upload/World_Energy_By_Country_And_Region_1965_to_2023.csv
upload/Deep-learning-in-insurance--Accuracy-and-model-int_2023_Expert-Systems-with-.pdf


ic| result['Name']: 'Deep-learning-in-insurance--Accuracy-and-model-int_2023_Expert-Systems-with-.pdf'
    result['Hash']: 'QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q'
ic| file_metadata_CID: 'QmY7xwK4beVPQLbeKxEAeStqxY69f7GzRhqTukXXsLMkMh'
ic| result['Name']: 'Jean-Baptiste_Perronneau_-_Magdaleine_Pinceloup_de_la_Grange,_née_de_Parseval.jpg'
    result['Hash']: 'QmVTTcqbGvYRn7n7uPYwk4vi8NdbeYvnZkX9MVT3byrAx2'
ic| file_metadata_CID: 'QmRAMLeY5eeV7HwexwcmP2SgnTghyHKvXXTQisipBvmD3E'
ic| result['Name']: 'bitcoin.pdf'
    result['Hash']: 'QmRA3NWM82ZGynMbYzAgYTSXCVM14Wx1RZ8fKP42G6gjgj'
ic| file_metadata_CID: 'QmUa22LAJeLd2Zf57bKgyLm9Ja9BDCfhmxXkxJzX55Jpvw'
ic| result['Name']: 'hello_world.py'
    result['Hash']:

upload/Jean-Baptiste_Perronneau_-_Magdaleine_Pinceloup_de_la_Grange,_née_de_Parseval.jpg
upload/bitcoin.pdf
upload/hello_world.py


 'QmeBTz4ZwyqPkPigpASZ3wJodBXzJRpeXQMwNcsWRqU5Jv'
ic| file_metadata_CID: 'QmWwZb9S8tE764UCs6Lvzfevjuv7wWjKbYQMqNpJmZba14'
